In [1]:
import os
import cv2
import pandas as pd

In [2]:
videos_folder = "../data/videos"
gt_folder = "../data/gt_annotations"
temp_folder = "../data/tmp/"

In [3]:
episode_names = []
video_paths = {}
for file in os.listdir(videos_folder):
    if file.endswith(".avi"):
        file_path = os.path.join(videos_folder, file)
        episode_name = file.split('.')[0]
        episode_names.append(episode_name)
        video_paths[episode_name] = file_path

gt_dfs = {}
for file in os.listdir(gt_folder):
    if file.endswith(".csv"):
        file_path = os.path.join(gt_folder, file)
        df = pd.read_csv(file_path, sep=";")
        episode_name = file.split('.')[0]
        assert episode_name in episode_names
        gt_dfs[episode_name] =  df

In [4]:
def load_video(path):
    cap = cv2.VideoCapture(path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    h = int(cap.get(cv2.CAP_PROP_FOURCC))
    codec = chr(h&0xff) + chr((h>>8)&0xff) + chr((h>>16)&0xff) + chr((h>>24)&0xff) 
    print(f"Loaded video with resolution {frame_width}x{frame_height}")
    print(f"FPS = {fps}, Frame Count = {frame_count}, Codec = {codec}")
    return cap, fps, frame_width, frame_height

In [5]:
episode = episode_names[0]
df = gt_dfs[episode]

codec = cv2.VideoWriter_fourcc('a', 'v', 'c', '1')
out_filepath = os.path.join(temp_folder, 'out.mp4')

In [6]:
video, fps, frame_width, frame_height = load_video(video_paths[episode])
out_video = cv2.VideoWriter(out_filepath, codec, fps, (frame_width, frame_height))

i = 0
while(True):
    if i%1000 == 0:
        print(i)
        if i>=2000:
            break
    i = i + 1
    ret, frame = video.read()
    frame_annotation = df[df.Frame_number == i]
    if (frame_annotation["Kermit"] == 1).all():
        frame = cv2.putText(frame, "Kermit", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 255, 0), 3, cv2.LINE_AA)
    if ret == True:
        out_video.write(frame)
    else:
        break 
video.release()
out_video.release()

Loaded video with resolution 720x544
FPS = 25, Frame Count = 38682, Codec = FMP4
0
1000
2000


In [7]:

from ipywidgets import Video
Video.from_file(out_filepath, width=320, height=320)

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free...', height='320', wid…

In [9]:
df[df["Kermit"] == 1]

,Video,Frame_number,Kermit,Pigs,Audio_Pigs,Cook,Audio_Cook,StatlerWaldorf,Audio_StatlerWaldorf,Audio_MissPiggy
970,211,970,1,0,0,0,0,0,0,0
971,211,971,1,0,0,0,0,0,0,0
972,211,972,1,0,0,0,0,0,0,0
973,211,973,1,0,0,0,0,0,0,0
974,211,974,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
37499,211,37499,1,0,0,0,0,0,0,0
37500,211,37500,1,0,0,0,0,0,0,0
37501,211,37501,1,0,0,0,0,0,0,0
37502,211,37502,1,0,0,0,0,0,0,0
